## PAIR PROGRAMMING ETL II

### Transformación I - Limpieza
---

In [2]:
import requests
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda


import ast

from datetime import datetime, timedelta

pd.options.display.max_columns = None

Tendréis que usar el csv attacks_limpieza_completa.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El **objetivo** de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [3]:
df= pd.read_csv('../files/attacks_limpieza_completa.csv', index_col=0)
df_clima = pd.read_csv('../files/clima_paises.csv', index_col=0)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

In [4]:
df_attacks= df[df['country'].isin(['usa','australia','new zealand','south africa', 'papua new guinea'])]

In [5]:
df_attacks['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [6]:
df_attacks.shape

(4335, 21)

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [7]:
df_clima.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country,highcloud,midcloud,lowcloud,rh_profile,wind_profile,msl_pressure,snow_depth
0,3,1,15,none,0,12,4,NaN,270,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",1026.0,0.0
1,6,1,15,none,0,12,3,NaN,15,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",1027.0,0.0


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [8]:
df_clima.dtypes

timepoint              int64
cloudcover             int64
lifted_index           int64
prec_type             object
prec_amount            int64
temp2m                 int64
rh2m                   int64
weather              float64
wind10m.direction      int64
wind10m.speed          int64
country               object
highcloud            float64
midcloud             float64
lowcloud             float64
rh_profile            object
wind_profile          object
msl_pressure         float64
snow_depth           float64
dtype: object

In [9]:
df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

In [10]:
x = df_clima['rh_profile'].apply(pd.Series)

In [11]:
x.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': -2}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 4}"
1,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"


In [12]:
for i in range(len(x.columns)): 
    

    # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)

In [13]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [14]:
y = df_clima['wind_profile'].apply(pd.Series)

In [15]:
for i in range(len(y.columns)): 
    

    # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "direction" + str(y[i].apply(pd.Series)["layer"][0]) 
    nombre2 = "speed" + str(y[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(y[i].apply(pd.Series)["direction"] )
    valores2= list(y[i].apply(pd.Series)["speed"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)
    df_clima.insert(i,nombre2,valores2)

In [17]:
df_clima.drop(['rh_profile','wind_profile'], axis=1, inplace=True)# Eliminamos las columnas que tienen las listas de diccionarios.

In [18]:
df_clima.shape

(320, 64)

In [41]:
df_clima = df_clima.groupby('country').mean()

/var/folders/db/hp0l5_js4yv8br5y2tc3frpm0000gn/T/ipykernel_43417/229230069.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima = df_clima.groupby('country').mean()


In [42]:
df_clima

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction200mb,direction250mb,direction300mb,direction350mb,direction400mb,direction450mb,direction500mb,direction550mb,direction600mb,direction650mb,direction700mb,direction750mb,direction800mb,direction850mb,direction900mb,direction950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,lifted_index,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,highcloud,midcloud,lowcloud,msl_pressure,snow_depth
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
australia,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,4.000000,4.031250,4.250000,4.593750,5.093750,5.671875,161.015625,151.875000,138.203125,123.203125,120.546875,121.015625,129.453125,133.906250,132.421875,136.484375,133.828125,136.250000,135.546875,129.609375,115.703125,121.406250,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.375000,2.171875,2.078125,1.546875,1.640625,4.125000,6.984375,97.5,3.703125,-3.750000,2.859375,25.890625,10.906250,NaN,122.968750,3.296875,-9999.0,-9999.0,-9999.0,1014.906250,0.0
new zealand,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,4.484375,4.796875,5.093750,5.406250,6.406250,7.500000,288.593750,276.484375,264.609375,254.843750,241.796875,233.515625,219.843750,216.328125,223.125000,218.984375,212.500000,194.765625,191.875000,178.359375,159.921875,157.890625,13.359375,12.906250,7.921875,5.296875,3.265625,2.671875,1.875000,1.703125,3.140625,4.640625,4.906250,5.109375,5.031250,4.046875,1.421875,-1.812500,97.5,6.734375,9.093750,3.203125,15.062500,10.859375,NaN,149.453125,3.359375,-9999.0,-9999.0,-9999.0,1015.500000,0.0
papua new guinea,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,4.062500,4.218750,4.265625,4.140625,4.687500,5.078125,221.562500,190.468750,220.859375,239.140625,243.281250,256.796875,260.859375,240.781250,196.953125,145.234375,79.843750,77.343750,80.390625,82.109375,82.031250,82.812500,13.578125,11.531250,9.343750,5.812500,2.531250,0.343750,1.296875,0.031250,-1.812500,-1.906250,0.296875,1.750000,2.218750,2.218750,2.125000,1.171875,97.5,3.546875,-0.671875,2.109375,25.890625,11.500000,NaN,84.375000,3.156250,-9999.0,-9999.0,-9999.0,1010.265625,0.0
south africa,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,3.843750,4.140625,4.578125,5.093750,5.359375,5.968750,208.437500,209.687500,212.812500,216.953125,219.218750,216.875000,215.000000,211.406250,200.468750,203.125000,202.968750,196.250000,181.484375,159.062500,142.812500,147.656250,12.328125,8.703125,6.546875,7.421875,8.812500,9.953125,9.984375,8.296875,5.625000,4.562500,4.687500,4.953125,5.656250,6.312500,7.593750,9.390625,97.5,7.031250,2.562500,1.781250,23.421875,10.156250,NaN,152.500000,2.296875,-9999.0,-9999.0,-9999.0,1019.484375,0.0
usa,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,4.968750,5.046875,5.078125,5.421875,5.640625,5.484375,248.828125,222.500000,235.390625,240.468750,260.234375,271.640625,263.671875,267.656250,261.015625,261.562500,267.187500,271.718750,279.921875,276.875000,230.468750,222.812500,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.281250,4.921875,5.046875,6.078125,6.656250,5.937500,5.734375,97.5,3.562500,13.265625,0.000000,12.718750,4.812500,NaN,203.750000,2.093750,-9999.0,-9999.0,-9999.0,1022.968750,0.0


    Ver que country esta como index y hay que sacarlo para poder mergearlo o concatenarlo.

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.